In [50]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brenth
import scipy.integrate as integrate

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

pi=4*np.arctan(1.)

In [51]:
def root(F, x1, x2, div, it):
    X=np.linspace(x1,x2, div)
    roots=np.array([])
    for i in range(div-1):
        if F(X[i])*F(X[i+1])<=0:
            p1=X[i]
            p2=X[i+1]
            r=brenth(F, p1, p2, maxiter=it)
            roots=np.append(roots, r)
    return roots

In [52]:
def maxmin(dF,d2F, x1, x2, div, it):
    roots=root(dF, x1, x2, div, it)
    Max=np.array([])
    Min=np.array([])
    Infl=np.array([])
    for r in roots:
        if d2F(r)>0: Min=np.append(Min, r)
        if d2F(r)<0: Max=np.append(Max, r)
        if d2F(r)==0: Infl=np.append(Infl, r)
    return Max,Min, Infl

In [53]:
def integral(F, X):
    l=len(X)
    Func=np.zeros(l)
    xmin=X[int(0.5*l)]
    for i in range(l):
        x=X[i]
        Func[i]=integrate.quad(F, xmin, x)[0]
    return Func

In [71]:
def zn(a,b,n,x):
    return (a/b)**n*np.sin(b**n*pi*x)
def Zp(x,p):
    r=0
    for i in range(p+1):
        r+= zn(a,b,i,x)
    return r/pi

def wn(a,b,n,x):
    return a**n*np.cos(b**n*pi*x)
def Wp(x,p):
    r=0
    for i in range(p+1):
        r+= wn(a,b,i,x)
    return r

def dwn(a,b,n,x):
    return a**n*b**n*np.sin(b**n*pi*x+0.5*pi)
def dWp(x,p):
    r=0
    for i in range(p+1):
        r+= dwn(a,b,i,x)
    return -pi*r

def d2wn(a,b,n,x):
    return (a*b**2)**n*np.cos(b**n*pi*x)
def d2Wp(x,p):
    r=0
    for i in range(p+1):
        r+= d2wn(a,b,i,x)
    return -pi**2*r

In [72]:
def dUeff(x):
    return (x+alpha*Wp(x,p))*(1.-2*(1.+alpha*dWp(x,p))/(g1**2*g2))

def T(x):
    return 1.-(x+alpha*Wp(x,p))**2/g1**2

def integrand(x):
    return dUeff(x)/T(x)

def LL(x):
    return x+alpha*Wp(x,p)+g1

def UL(x):
    return x+alpha*Wp(x,p)-g1

def Ueff(x):
    return 0.5*x**2+alpha*Zp(x,p)+T(x)/g2

def d2Ueff(x):
    a1=1+alpha*dWp(x,p)
    a2=x+alpha*Wp(x,p)
    a3=-2*alpha*d2Wp(x,p)/g2/g1**2
    return a1*(1-2*a1/g2/g1**2)+a2*a3

In [60]:
p=2

a=0.25
b=np.rint((1.+1.5*pi)/a) #no siempre es impar
b=b+np.mod(b+1,2)

print a*b>=1+1.5*pi

sp=(1.-a**p)/(1.-a)

True


In [83]:
g1=0.5
g2=0.5
#gc=0.5/g1**2
#g2=0.01*gc
alpha=0.1

In [84]:
L1=-g1-alpha*sp
L2=-g1+alpha*sp

U1=g1-alpha*sp
U2=g1+alpha*sp

In [85]:
xmin=max(root(T, L1,L2, 10000, 300))
xmax=min(root(T, U1,U2, 10000, 300))
x=np.linspace(xmin,xmax, 2000)

In [87]:
y=dUeff(x)
plt.plot(x,y)
plt.show()

In [91]:
r=root(dUeff, xmin, xmax, 10000, 300)

In [94]:
X=np.linspace(xmin*99, xmax*.99, 1000)
X=np.concatenate((X,r), axis=0)
X=np.sort(X)

inte=integral(integrand, X)
inte=-g2*inte
prob=np.exp(inte)
prob[prob==np.nan]=0
Z=np.trapz(prob, X)

/home/brawantonio/anaconda2/lib/python2.7/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)
/home/brawantonio/anaconda2/lib/python2.7/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)
/home/brawantonio/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: overflow encountered in exp


In [105]:
plt.plot(X[prob<1e5], prob[prob<1e5])
plt.show()